# Global Variables

In [44]:
import pandas as pd
import numpy as np
import warnings
import time
import datetime
import logging
#warnings.filterwarnings('ignore')

exchange = 'NSE'
symbol = "HDFC"

today = datetime.datetime.now()
#today = datetime.datetime(2019, 4, 8, 15, 45, 0)

#configuration for downloading data
days = 2 * 365
years = 5
toDate = today
fromDate = toDate - datetime.timedelta(days = days)
freq = '1min'

mode = "Backtesting" # Backtesting/Live
offline = True
if mode =="Live":
    offline = False

KiteAPIKey = "b2w0sfnr1zr92nxm"
KiteAPISecret = "jtga2mp2e5fn29h8w0pe2kb722g3dh1q"

from enum import Enum
class S(Enum):
    BUY = 1
    SELL = 2
    HOLD = 0

%run "KiteConnect_Library.ipynb"    

# Kite - OAuth Login

In [2]:
from kiteconnect import KiteConnect
kite = KiteConnect(api_key=KiteAPIKey)
kite.set_access_token('mQXlKzqI34Ty6gYN36FqE93OP00DAupl')

In [3]:
from kiteconnect import KiteConnect
import platform
from selenium import webdriver
import re

#logging.basicConfig(level=logging.DEBUG)
if offline != True:
    kite = KiteConnect(api_key=KiteAPIKey)
    
if mode == "Live":
    #kite.login_url()

    if platform.system() == "Windows":
        driver = webdriver.Chrome("./automation/chromedriver.exe")
    else:
        driver = webdriver.Chrome("./automation/chromedriver")
    driver.implicitly_wait(60)
    driver.get(kite.login_url())

In [4]:
access_token = ""
if mode == "Live":
    # Redirect the user to the login url obtained
    # from kite.login_url(), and receive the request_token
    # from the registered redirect url after the login flow.
    # Once you have the request_token, obtain the access_token
    # as follows.
    request_token = re.findall("request_token=(.*)&action",driver.current_url)

    data = kite.generate_session(request_token[0], api_secret=KiteAPISecret)
    access_token = data["access_token"]
    kite.set_access_token(access_token)
    print(access_token)
    driver.close()
elif offline != True:
    access_token = "zvyL9tcnN4ANGJmP3tqR7ty5AFaFynPk"
    kite.set_access_token(access_token)

# Download NSE Stock Master List

In [5]:
if offline != True:
    instruments_df = getInstruments(exchange)
    instruments_df.to_hdf('kite_data/kite_data.h5', key=exchange, mode='w')

instruments_df = pd.read_hdf('kite_data/kite_data.h5', key=exchange, mode='r')

EQSYMBOL = lambda x:instruments_df[instruments_df['instrument_token']==x].index[0]
EQTOKEN = lambda x:instruments_df.loc[x,'instrument_token']

# Filter Stocks - portfolio maker

In [6]:
nifty50 = pd.read_csv("data/ind_nifty50list.csv")
niftynext50 = pd.read_csv("data/ind_niftynext50list.csv")
midcap50 = pd.read_csv("data/ind_niftymidcap50list.csv")

downloadlist = nifty50['Symbol']
industry = niftynext50['Industry'].unique()

In [7]:
portfolio = pd.DataFrame(['TCS', 'RIIL', 'HDFC'])
portfolioToken = portfolio[0].apply(EQTOKEN)

# Download Historical Data - Equity

In [8]:
def portfolioDownload(stocklist, toDate):
    stocklist_df = pd.DataFrame()
    for index, row in stocklist.iterrows():
        symbol = row[0]
        print("Downloading data for: "+symbol)
        temp_data = downloadData(symbol,  toDate - dt.timedelta(days = 1), toDate)
        temp_data['symbol'] = symbol
        temp_data.set_index(['symbol',temp_data.index], inplace=True)
        #print(temp_data)
        stocklist_df = stocklist_df.append(temp_data)
    
    #print(stocklist_df)
    return stocklist_df    

if offline != True:
    portfolio_df = pd.DataFrame()
    portfolio_df = portfolioDownload(portfolio, today)
    portfolio_df.to_hdf('kite_data/kite_data.h5', key="portfolio",mode='a')

#portfolio_df.loc['HDFC']

### Batch Downloader

In [9]:
def batchDownload(downloadlist):
    for symbol in downloadlist:
        print("Downloading data for: "+symbol)

        raw_data = downloadData(symbol, fromDate, toDate)
        raw_data_day = downloadData(symbol,  toDate - dt.timedelta(days = years * 365),toDate, freq="day")

        raw_data_day.to_hdf('kite_data/kite_data.h5', key=symbol+"day",mode='a')
        raw_data.to_hdf('kite_data/kite_data.h5', key=symbol,mode='a')

## Incremental Download

In [10]:
def incrementalDownload(downloadlist):
    for symbol in downloadlist:
        print("Downloading data for: "+symbol)
        tempData = pd.read_hdf('kite_data/kite_data.h5', key=symbol,mode='r')
        fromDate = tempData.index[-1]
        toDate = datetime.datetime.now()
        raw_data = downloadData(symbol, fromDate, toDate)
        tempData = tempData.append(raw_data)
        tempData.to_hdf('kite_data/kite_data.h5', key=symbol,mode='a')
        
        tempData = pd.read_hdf('kite_data/kite_data.h5', key=symbol+"day",mode='r')
        fromDate = tempData.index[-1]
        toDate = datetime.datetime.now()
        raw_data = downloadData(symbol, fromDate, toDate, freq="day")
        
        tempData.to_hdf('kite_data/kite_data.h5', key=symbol+"day",mode='a')
    
    

### Historical Data storage strategy

* minute level data and day level ohlc data is stored in the hd5
* symbol name is used as a key in the hd5 file system

### Tick Data storage strategy
- Tick data should not be merged with the downloaded historical data
- At the start of the trading session, last 60 candles from the historical data is fetched and stored in the dataframe for holding live data
- Two DataFrames ares created for storing live data: 1 for storing timestamp and LTP, another for OHLC
- OHLC data is created by grouping data first based on stock symbol and then based on timestamp(seconds and miliseconds are ignored)
- Streaming data from all the stocks in portfolio are strored in a single table. During post-procession subset of the master table sliced based on symbol is used
- Streaming data is resampled for minute frequency every minute to convert it to OHLC data which is stored in another dataframe
- Symbol is used as a key for the streaming OHLC dataframe storage

# Load Data from disk

In [12]:
if mode=="Live":
    raw_data = pd.read_hdf('kite_data/kite_data.h5', key="portfolio",mode='r')
else:
    raw_data = pd.read_hdf('kite_data/kite_data.h5', key="portfolio",mode='r')
    #raw_data_day = pd.read_hdf('kite_data/kite_data.h5', key="TCS"+"day",mode='r')

# Batch Analysis mode

In [47]:
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objs as go
from plotly import tools
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot


#symbol = portfolio.iloc[2,0]
#print(symbol)

fig = ""
noofcandles = 50
def batchAnalysis():
    global fig, noofcandles
    noofcandles = 50
    
    for index, row in portfolio.iterrows():
        fig = ""
        symbol = row.loc[0]
        print(raw_data.loc[symbol].index[-1*noofcandles])
        
        yMin = raw_data.loc[symbol].iloc[-1*noofcandles:-1]['low'].min()-10
        yMax = raw_data.loc[symbol].iloc[-1*noofcandles:-1]['high'].max()
        
        xMin = raw_data.loc[symbol].index[-1*noofcandles]
        xMax = raw_data.loc[symbol].index[-1]
        fig = createPlot(symbol)
        fig['layout']['yaxis']['range'] = [yMin, yMax]
        fig['layout']['yaxis']['range'] = [xMin, xMax]

        temp_data = resample2(raw_data.loc[symbol],"1min").dropna()[-20*noofcandles:-1]
        temp_data = candlestick(temp_data,1,True)
        temp_data = bbands(temp_data,1, True)

        temp_data = macd(temp_data,3,True)
        temp_data = rsi(temp_data,4, True)
        temp_data = aroon(temp_data,5, True)
        obv(temp_data,2, True)
        
        a = algoTrade(temp_data)
        buy, sell = a.tradeDecision()
        buy_df = annotateBuySell(buy, "Buy")
        sell_df = annotateBuySell(sell, "Sell")
        
        plotData(symbol)
        #input("Press Enter to continue")
    

batchAnalysis()

2019-04-10 15:02:00
This is the format of your plot grid:
[ (1,1) x1,y1 ]
[ (2,1) x1,y2 ]
[ (3,1) x1,y3 ]
[ (4,1) x1,y4 ]
[ (5,1) x1,y5 ]



2019-04-10 14:58:00
This is the format of your plot grid:
[ (1,1) x1,y1 ]
[ (2,1) x1,y2 ]
[ (3,1) x1,y3 ]
[ (4,1) x1,y4 ]
[ (5,1) x1,y5 ]



2019-04-10 15:01:00
This is the format of your plot grid:
[ (1,1) x1,y1 ]
[ (2,1) x1,y2 ]
[ (3,1) x1,y3 ]
[ (4,1) x1,y4 ]
[ (5,1) x1,y5 ]



# Indicators Initialization

In [18]:
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objs as go
from plotly import tools
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

noofcandles = 50

symbol = portfolio.iloc[0,0]

yMin = raw_data.loc[symbol].iloc[-1*noofcandles:-1]['low'].min()-10
yMax = raw_data.loc[symbol].iloc[-1*noofcandles:-1]['high'].max()
xMin = raw_data.loc[symbol].index[-1*noofcandles]
xMax = raw_data.loc[symbol].index[-1]

In [19]:
symbol

'TCS'

# Algorithm

In [46]:
#pivotPoint(raw_data, pos=1, plot=False)
#raw_data = calculateStats(raw_data)
#raw_data = emasma(raw_data)
#raw_data = adx(raw_data)
#raw_data = detectCDPattern(raw_data,strPlot="hammer", plot=True)
#raw_data = stoch(raw_data)

# ====== Tradescript Wrapper =======
# Variables
OPEN = 0
CLOSE = 0
HIGH = 0
LOW = 0
VOLUME = 0
BBT = 0
BBM = 0
BBB = 0
AroonUp = 0
AroonDown = 0

UP = 0
DOWN = 1

# Methods
REF = lambda df, i: df.shift(i)
TREND_UP = lambda a, days: ROC(a,days) >= 0.01
TREND_DOWN = lambda a, days: ROC(a,days) <= -0.01
CROSSOVER = lambda a, b: (REF(a,1)<=REF(b,1)) & (a > b)

class algoTrade:
    
    def __init__(s, price):
        #logging.debug(price.tail(10))
        global OPEN, CLOSE, HIGH, LOW, VOLUME, BBT, BBM, BBB, AroonDown, AroonUp
        OPEN = price['open']
        CLOSE = price['close']
        HIGH = price['high']
        LOW = price['low']
        VOLUME = price['volume']
        BBT, BBM, BBB = BBANDS( CLOSE, 20,2,2,1)
        AroonDown, AroonUp = AROON(HIGH, LOW, 25)
    
    # Long Strategies
    def long_bb(self):
        return pd.DataFrame( (REF(CLOSE, 1) < REF(BBB, 1)) & (CLOSE > BBB), columns=["buy"] )
    
    def long_bull_engulf_ema(self):
        return pd.DataFrame( (EMA(CLOSE, 9) < EMA(CLOSE, 21)) & (CDLENGULFING(OPEN, HIGH, LOW, CLOSE) == 100) ,columns =["buy"])
    
    def long_hammer(self):
        return pd.DataFrame(TREND_DOWN(CLOSE, 10) & (CDLHAMMER(OPEN, HIGH, LOW, CLOSE) != 0), columns=["buy"])
    
    def long_ichimoku(self):
        C = ( SMA(HIGH, 9) + SMA(LOW, 9) )/2
        D = ( SMA(HIGH, 26) + SMA(LOW, 26) )/2
        A = (C+D)/2
        B = (SMA(HIGH,52)+SMA(LOW,52))/2
        
        return pd.DataFrame( CROSSOVER(A,B) ,columns=["buy"])
    
    def long_aroon(self):
        return pd.DataFrame((AroonUp > 50) & (AroonDown<50), columns=["buy"])
        
    
    # Short Strategies
    def short_bb(self):
        return pd.DataFrame((REF(CLOSE,1) > REF(BBT,1)) & (CLOSE<BBT), columns=["sell"])
    
    def short_bear_engulf(self):
        return pd.DataFrame(TREND_UP(CLOSE,10) & (CDLENGULFING(OPEN, HIGH, LOW, CLOSE) == -100),columns=["sell"])
    
    def short_hanging_man(self):
        return pd.DataFrame(
            TREND_UP(CLOSE, 10) &
            (CDLHANGINGMAN(OPEN, HIGH, LOW, CLOSE) == -100)
            , columns=["sell"])
    
    def short_bear_kicking(self):
        return pd.DataFrame(CDLKICKING(OPEN, HIGH, LOW, CLOSE) != 0, columns=["sell"])
        
    def short_shooting_star(self):
        return pd.DataFrame( TREND_UP(CLOSE, 5) & CDLSHOOTINGSTAR(OPEN, HIGH, LOW, CLOSE) != 0, columns=["sell"])
    
    def short_ichimoku(self):
        C = ( SMA(HIGH, 9) + SMA(LOW, 9) )/2
        D = ( SMA(HIGH, 26) + SMA(LOW, 26) )/2
        A = (C+D)/2
        B = (SMA(HIGH,52)+SMA(LOW,52))/2
        
        return pd.DataFrame( CROSSOVER(B,A) ,columns=["sell"])
    
    def short_aroon(self):
        return pd.DataFrame((AroonUp < 50) & (AroonDown > 50), columns=["sell"])
    
    def tradeDecision(self):
        buy  = self.long_bull_engulf_ema() | self.long_bb() | self.long_ichimoku()
        #buy = self.long_aroon()
        sell = self.short_bear_engulf() | self.short_bb() | self.short_hanging_man() |  self.short_ichimoku()
        #sell = self.short_aroon()
        
        buy['low'] = LOW
        sell['high'] = HIGH
        buy['close'] = sell['close'] = CLOSE

        buy = buy[buy['buy']]
        sell = sell[sell['sell']]
        return (buy,sell)
    
    def tradeRecommendation(self):
        buy, sell = self.tradeDecision()
        timeNow = dt.datetime.now().replace(second=0, microsecond=0)
        delT1 = timeNow - datetime.timedelta(minutes = 1)
        delT2 = timeNow - datetime.timedelta(minutes = 2)
        logging.debug(str(timeNow))
        logging.debug(buy.tail(3))
        logging.debug(sell.tail(3))
        if (delT1 == buy.index[-1]) | (delT2 == buy.index[-1]) :
            return "buy,"+str(timeNow) +","+str(buy.iloc[-1]['low'])+","+str(buy.iloc[-1]['close'])+"\n"
        elif (delT1 == sell.index[-1]) | (delT2 == sell.index[-1]) :  
            return "sell,"+str(timeNow) +","+str(sell.iloc[-1]['high'])+","+str(sell.iloc[-1]['close'])+"\n"

a = algoTrade(temp_data)
buy, sell = a.tradeDecision()
a.tradeRecommendation()
buy_df = annotateBuySell(buy, "Buy")
sell_df = annotateBuySell(sell, "Sell")


NameError: name 'temp_data' is not defined

In [21]:
plotData()

Press Enter to continue


# Order Management

In [25]:
def placeorder(): 
    
    if False:
        # Place an order
        try:
            order_id = kite.place_order(tradingsymbol="INFY",
                                        exchange=kite.EXCHANGE_NSE,
                                        transaction_type=kite.TRANSACTION_TYPE_BUY,
                                        quantity=1,
                                        order_type=kite.ORDER_TYPE_MARKET,
                                        product=kite.PRODUCT_NRML)

            logging.info("Order placed. ID is: {}".format(order_id))
        except Exception as e:
            logging.info("Order placement failed: {}".format(e.message))


def getOrders():    
    # Fetch all orders
    kite.orders()



# WebSocket: Live Tick Handler

In [58]:
import datetime

def resample(ws, freq="1min"):
    F = open("kite_data/recommendation.csv","a") 
    
    logging.debug(str(ws.prevtimeStamp)+": In resampler function")
    
    if ws.LiveStream.empty:
        logging.debug(str(ws.prevtimeStamp)+": Empty dataframe, Exiting resampler")
        return
    
    
    logging.debug(ws.LiveStream.head())
    
  
    LiveStream2 = ws.LiveStream.groupby(['symbol','date']).agg({'price':['first','max','min','last'], 'volume':['last']})
    LiveStream2.columns = LiveStream2.columns.droplevel()
    LiveStream2.columns = ['open', 'high','low','close', 'volume']

    for index, data in LiveStream2.groupby(level=0):
        #print(index)
        sampled = data.loc[index].resample(freq).agg({'open':{'open':'first'},'high':{'high':'max'},'low':{'low':'min'},'close':{'close':'last'},'volume':{'volume':'last'}})
        sampled.columns = sampled.columns.droplevel()

        #print(sampled)
        sampled['symbol'] = index
        sampled.set_index(['symbol',sampled.index], inplace=True)

        ws.LiveStreamOHLC = ws.LiveStreamOHLC.append(sampled)

    ws.LiveStreamOHLC.to_csv("kite_data/livestreamohlc.csv", mode='a')

    #print(ws.LiveStreamOHLC.loc['HDFC'].tail(1))
    a = algoTrade(ws.LiveStreamOHLC.loc['HDFC'])
    reco_df = a.tradeRecommendation()

    try:
        F.write(reco_df)
    except:
        logging.critical("Not able to resample and analyse data")
    finally:
        F.close()
        



def ticksHandler(ws, ticks):
    timeStamp = dt.datetime.now().replace(second=0, microsecond=0)
    tick_df = pd.DataFrame(ticks)
    
    #logging.debug("Prev: "+str(ws.prevtimeStamp)+"\nNow: "+str(timeStamp))
    
    try:
        tick_df.loc[tick_df['timestamp'].isna(), 'timestamp'] = timeStamp
        tick_df = tick_df[['timestamp','instrument_token','last_price','volume']]
        tick_df.instrument_token = tick_df.instrument_token.apply(EQSYMBOL)
        tick_df.columns = ['date','symbol','price','volume']
        tick_df.set_index(['symbol','date'], inplace=True)
    except:
        print("Exception: ticksHandler")
    
    if(ws.prevtimeStamp != timeStamp):
        #logging.debug(tick_df)
        ws.prevtimeStamp = timeStamp
        resample(ws)
    
    ws.LiveStream = ws.LiveStream.append(tick_df)
    ws.LiveStream.to_csv("kite_data/livestream.csv", mode='a')


def initTrade(ws):  
    ws.prevtimeStamp = ""
    toDate = datetime.datetime.now()
    ws.LiveStream = pd.DataFrame()
    ws.LiveStreamOHLC = pd.DataFrame()
    ws.LiveStreamOHLC = portfolioDownload(portfolio, toDate)
    

In [59]:
#!python
import logging
import multiprocessing
from kiteconnect import KiteTicker
import copy 

#logging.basicConfig(level=logging.CRITICAL)
logging.basicConfig(filename="log/live_log.log", mode="w", level=logging.DEBUG)


jobs = []


# Initialise
kws = KiteTicker(KiteAPIKey, kite.access_token)

def on_ticks(ws, ticks):
    # Callback to receive ticks.
    #logging.debug("Ticks: {}".format(ticks))
    #count = count + 1

    #print(ticks)
    ticksHandler(ws, ticks)
    #jobs.append(p)
    #p.start()


def on_connect(ws, response):
    initTrade(ws)
    
    # Callback on successful connect.
    # Subscribe to a list of instrument_tokens (RELIANCE and ACC here).
    ws.subscribe(portfolioToken.tolist())

    # Set RELIANCE to tick in `full` mode.
    # MODE_LTP, MODE_QUOTE, or MODE_FULL

    ws.set_mode(ws.MODE_FULL, portfolioToken.tolist()) 
    #ws.set_mode(ws.MODE_LTP, [225537, 3861249]) 
    #ws.set_mode(ws.MODE_MODE_QUOTE, [2714625,779521]) 

def on_close(ws, code, reason):
    # On connection close stop the main loop
    # Reconnection will not happen after executing `ws.stop()`
    ws.stop()

# Assign the callbacks.
kws.on_ticks = on_ticks
kws.on_connect = on_connect
#kws.on_close = on_close

# Infinite loop on the main thread. Nothing after this will run.
# You have to use the pre-defined callbacks to manage subscriptions.

#p = multiprocessing.Process(target=kws.connect)
#jobs.append(p)
#p.start()


In [67]:
kws.connect(threaded=True)

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443


DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /instruments/historical/340481/minute?instrument_token=340481&from=2019-04-09+11%3A10%3A04&to=2019-04-10+11%3A10%3A04&interval=minute&continuous=0 HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443


DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /instruments/historical/745473/minute?instrument_token=745473&from=2019-04-09+11%3A10%3A04&to=2019-04-10+11%3A10%3A04&interval=minute&continuous=0 HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.kite.trade:443


DEBUG:urllib3.connectionpool:https://api.kite.trade:443 "GET /instruments/historical/2953217/minute?instrument_token=2953217&from=2019-04-09+11%3A10%3A04&to=2019-04-10+11%3A10%3A04&interval=minute&continuous=0 HTTP/1.1" 200 None
DEBUG:root:2019-04-10 11:10:00: In resampler function
DEBUG:root:2019-04-10 11:10:00: Empty dataframe, Exiting resampler
DEBUG:root:2019-04-10 11:11:00: In resampler function
DEBUG:root:                              price   volume
symbol date                                 
RIIL   2019-04-10 11:09:58   353.00   263148
HDFC   2019-04-10 11:10:07  2051.10   818763
TCS    2019-04-10 11:10:07  2055.40  1275446
HDFC   2019-04-10 11:10:08  2051.15   818764
TCS    2019-04-10 11:10:07  2055.40  1275607
C:\Users\suhan\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:22: FutureWarning:

using a dict with renaming is deprecated and will be removed in a future version

DEBUG:root:                       close     high      low     open  volume
date

DEBUG:root:                      buy      low    close
date                                       
2019-04-10 11:02:00  True  2050.95  2051.30
2019-04-10 11:10:00  True  2050.15  2050.15
2019-04-10 11:14:00  True  2046.35  2046.95
DEBUG:root:                     sell     high    close
date                                       
2019-04-10 10:38:00  True  2056.80  2054.35
2019-04-10 10:59:00  True  2054.35  2052.35
2019-04-10 11:03:00  True  2052.65  2052.00
CRITICAL:root:Not able to resample and analyse data
DEBUG:root:2019-04-10 11:16:00: In resampler function
DEBUG:root:                              price   volume
symbol date                                 
RIIL   2019-04-10 11:09:58   353.00   263148
HDFC   2019-04-10 11:10:07  2051.10   818763
TCS    2019-04-10 11:10:07  2055.40  1275446
HDFC   2019-04-10 11:10:08  2051.15   818764
TCS    2019-04-10 11:10:07  2055.40  1275607
DEBUG:root:                       close     high      low     open  volume
date                           

DEBUG:root:                      buy      low    close
date                                       
2019-04-10 11:02:00  True  2050.95  2051.30
2019-04-10 11:10:00  True  2050.15  2050.15
2019-04-10 11:14:00  True  2046.35  2046.95
DEBUG:root:                     sell     high    close
date                                       
2019-04-10 10:38:00  True  2056.80  2054.35
2019-04-10 10:59:00  True  2054.35  2052.35
2019-04-10 11:03:00  True  2052.65  2052.00
CRITICAL:root:Not able to resample and analyse data
DEBUG:root:2019-04-10 11:21:00: In resampler function
DEBUG:root:                              price   volume
symbol date                                 
RIIL   2019-04-10 11:09:58   353.00   263148
HDFC   2019-04-10 11:10:07  2051.10   818763
TCS    2019-04-10 11:10:07  2055.40  1275446
HDFC   2019-04-10 11:10:08  2051.15   818764
TCS    2019-04-10 11:10:07  2055.40  1275607
DEBUG:root:                       close     high      low     open  volume
date                           

DEBUG:root:                      buy      low    close
date                                       
2019-04-10 11:02:00  True  2050.95  2051.30
2019-04-10 11:10:00  True  2050.15  2050.15
2019-04-10 11:14:00  True  2046.35  2046.95
DEBUG:root:                     sell     high    close
date                                       
2019-04-10 10:38:00  True  2056.80  2054.35
2019-04-10 10:59:00  True  2054.35  2052.35
2019-04-10 11:03:00  True  2052.65  2052.00
CRITICAL:root:Not able to resample and analyse data
DEBUG:root:2019-04-10 11:26:00: In resampler function
DEBUG:root:                              price   volume
symbol date                                 
RIIL   2019-04-10 11:09:58   353.00   263148
HDFC   2019-04-10 11:10:07  2051.10   818763
TCS    2019-04-10 11:10:07  2055.40  1275446
HDFC   2019-04-10 11:10:08  2051.15   818764
TCS    2019-04-10 11:10:07  2055.40  1275607
DEBUG:root:                       close     high      low     open  volume
date                           

DEBUG:root:                      buy      low    close
date                                       
2019-04-10 11:02:00  True  2050.95  2051.30
2019-04-10 11:10:00  True  2050.15  2050.15
2019-04-10 11:14:00  True  2046.35  2046.95
DEBUG:root:                     sell     high    close
date                                       
2019-04-10 10:38:00  True  2056.80  2054.35
2019-04-10 10:59:00  True  2054.35  2052.35
2019-04-10 11:03:00  True  2052.65  2052.00
CRITICAL:root:Not able to resample and analyse data
DEBUG:root:2019-04-10 11:31:00: In resampler function
DEBUG:root:                              price   volume
symbol date                                 
RIIL   2019-04-10 11:09:58   353.00   263148
HDFC   2019-04-10 11:10:07  2051.10   818763
TCS    2019-04-10 11:10:07  2055.40  1275446
HDFC   2019-04-10 11:10:08  2051.15   818764
TCS    2019-04-10 11:10:07  2055.40  1275607
DEBUG:root:                       close     high      low     open  volume
date                           

In [79]:
kws.close()
if kws.is_connected() == True:
    kws.close()
    print(kws.is_connected())

ERROR:kiteconnect.ticker:Connection closed: None - None


In [65]:
kws.is_connected()

True